In [1]:
! pip3 install idx2numpy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import pandas as pd
import numpy as np
import seaborn 
import idx2numpy
import gzip
import cv2
from matplotlib import pyplot
from urllib.request import urlretrieve
from sklearn.manifold import TSNE
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as skm
from sklearn.preprocessing import StandardScaler

*Run these commands to download the MNIST zip files*

In [3]:
# ! curl -O http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
# ! curl -O http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
# ! curl -O http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
# ! curl -O http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz

In [4]:
train_images = gzip.open('train-images-idx3-ubyte.gz','r')
train_labels = gzip.open('train-labels-idx1-ubyte.gz','r')
test_images = gzip.open('t10k-images-idx3-ubyte.gz','r')
test_labels = gzip.open('t10k-labels-idx1-ubyte.gz','r')
mnist_train = idx2numpy.convert_from_file(train_images)
mnist_train_labels = idx2numpy.convert_from_file(train_labels)
mnist_test = idx2numpy.convert_from_file(test_images)
mnist_test_labels = idx2numpy.convert_from_file(test_labels)

In [5]:
def flatten(dataset):
  nimages, nfeatures = dataset.shape
  lis = np.linspace(1, nfeatures, nfeatures, dtype=np.int64)
  lis = lis.tolist()
  lis = [str(x) for x in lis]
  dataset_flat = dataset.reshape((nimages, nfeatures))
  dataset_df = pd.DataFrame(dataset_flat, columns=lis)
  return dataset_df

In [6]:
scaler = StandardScaler()
scaler.fit(mnist_train.reshape(mnist_train.shape[0], -1))
mnist_test = scaler.transform(mnist_test.reshape(mnist_test.shape[0], -1))
mnist_train = scaler.transform(mnist_train.reshape(mnist_train.shape[0], -1))
mnist_test_flat = flatten(mnist_test)
mnist_train_flat = flatten(mnist_train)

In [7]:
mnist_train_flat.head()

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033171,-0.022463,-0.016069,-0.011432,-0.009007,-0.00577,0.0,0.0,0.0,0.0


In [8]:
mnist_train_labels.shape

(60000,)

(a) One-vs-One fashion of training the model. Logistic regression without any penalty and saga as the solver is used as the estimator.

In [9]:
ovo_clf = OneVsOneClassifier(LogisticRegression(random_state=2, solver='saga', penalty='none'))
ovo_clf.fit(mnist_train_flat, mnist_train_labels)
y_pred_ovo = ovo_clf.predict(mnist_test_flat)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

In [10]:
print('Metrics for One-vs-One Logistic Regression without regularization on test dataset')
print('Accuracy: ', skm.accuracy_score(mnist_test_labels, y_pred_ovo))
print('Precision: ', skm.precision_score(mnist_test_labels, y_pred_ovo, average='micro'))
print('Recall: ', skm.recall_score(mnist_test_labels, y_pred_ovo, average='micro'))
print('F1 Score: ', skm.f1_score(mnist_test_labels, y_pred_ovo, average='micro'))
print('Classification Report:\n', skm.classification_report(mnist_test_labels, y_pred_ovo))

Metrics for One-vs-One Logistic Regression without regularization on test dataset
Accuracy:  0.9347
Precision:  0.9347
Recall:  0.9347
F1 Score:  0.9347
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.97       980
           1       0.96      0.98      0.97      1135
           2       0.94      0.91      0.93      1032
           3       0.92      0.93      0.92      1010
           4       0.94      0.96      0.95       982
           5       0.91      0.89      0.90       892
           6       0.95      0.96      0.95       958
           7       0.94      0.92      0.93      1028
           8       0.91      0.89      0.90       974
           9       0.93      0.92      0.92      1009

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



(a) One-vs-One fashion of training the model. Logistic regression with l2 penalty and saga as the solver is used as the estimator.

In [11]:
ovo_clf_l2 = OneVsOneClassifier(LogisticRegression(random_state=2, solver='saga', penalty='l2'))
ovo_clf_l2.fit(mnist_train_flat, mnist_train_labels)
y_pred_ovo_l2 = ovo_clf_l2.predict(mnist_test_flat)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

In [12]:
print('Metrics for One-vs-One Logistic Regression with L2 regularization on test dataset')
print('Accuracy: ', skm.accuracy_score(mnist_test_labels, y_pred_ovo_l2))
print('Precision: ', skm.precision_score(mnist_test_labels, y_pred_ovo_l2, average='micro'))
print('Recall: ', skm.recall_score(mnist_test_labels, y_pred_ovo_l2, average='micro'))
print('F1 Score: ', skm.f1_score(mnist_test_labels, y_pred_ovo_l2, average='micro'))
print('Classification Report:\n', skm.classification_report(mnist_test_labels, y_pred_ovo_l2))

Metrics for One-vs-One Logistic Regression with L2 regularization on test dataset
Accuracy:  0.9347
Precision:  0.9347
Recall:  0.9347
F1 Score:  0.9347
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.97       980
           1       0.96      0.98      0.97      1135
           2       0.94      0.91      0.93      1032
           3       0.92      0.93      0.92      1010
           4       0.94      0.96      0.95       982
           5       0.91      0.89      0.90       892
           6       0.95      0.96      0.95       958
           7       0.94      0.92      0.93      1028
           8       0.91      0.89      0.90       974
           9       0.93      0.92      0.92      1009

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



(b) One-vs-Rest fashion of training the model. Logistic regression without penalty and saga as the solver is used as the estimator here.

In [13]:
ovr_clf = OneVsRestClassifier(LogisticRegression(random_state=2, solver='saga', penalty='none'))
ovr_clf.fit(mnist_train_flat, mnist_train_labels)
y_pred_ovr = ovr_clf.predict(mnist_test_flat)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

In [14]:
print('Metrics for One-vs-Rest Logistic Regression without regularization on test dataset')
print('Accuracy: ', skm.accuracy_score(mnist_test_labels, y_pred_ovr ))
print('Precision: ', skm.precision_score(mnist_test_labels, y_pred_ovr, average='micro'))
print('Recall: ', skm.recall_score(mnist_test_labels, y_pred_ovr, average='micro'))
print('F1 Score: ', skm.f1_score(mnist_test_labels, y_pred_ovr, average='micro'))
print('Classification Report:\n', skm.classification_report(mnist_test_labels, y_pred_ovr))

Metrics for One-vs-Rest Logistic Regression without regularization on test dataset
Accuracy:  0.9028
Precision:  0.9028
Recall:  0.9028
F1 Score:  0.9028
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95       980
           1       0.91      0.97      0.94      1135
           2       0.93      0.87      0.90      1032
           3       0.91      0.90      0.90      1010
           4       0.88      0.93      0.90       982
           5       0.88      0.84      0.86       892
           6       0.92      0.94      0.93       958
           7       0.89      0.90      0.90      1028
           8       0.87      0.84      0.85       974
           9       0.90      0.85      0.87      1009

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



(b) One-vs-Rest fashion of training the model. Logistic regression with l2 penalty and saga as the solver is used as the estimator here.

In [15]:
ovr_clf_l2 = OneVsRestClassifier(LogisticRegression(random_state=2, solver='saga', penalty='l2'))
ovr_clf_l2.fit(mnist_train_flat, mnist_train_labels)
y_pred_ovr_l2 = ovr_clf_l2.predict(mnist_test_flat)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

In [16]:
print('Metrics for One-vs-Rest Logistic Regression with l2 regularization on test dataset')
print('Accuracy: ', skm.accuracy_score(mnist_test_labels, y_pred_ovr_l2))
print('Precision: ', skm.precision_score(mnist_test_labels, y_pred_ovr_l2, average='micro'))
print('Recall: ', skm.recall_score(mnist_test_labels, y_pred_ovr_l2, average='micro'))
print('F1 Score: ', skm.f1_score(mnist_test_labels, y_pred_ovr_l2, average='micro'))
print('Classification Report:\n', skm.classification_report(mnist_test_labels, y_pred_ovr_l2))

Metrics for One-vs-Rest Logistic Regression with l2 regularization on test dataset
Accuracy:  0.9029
Precision:  0.9029
Recall:  0.9029
F1 Score:  0.9028999999999999
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95       980
           1       0.91      0.97      0.94      1135
           2       0.93      0.87      0.90      1032
           3       0.91      0.90      0.90      1010
           4       0.88      0.93      0.90       982
           5       0.88      0.84      0.86       892
           6       0.92      0.94      0.93       958
           7       0.89      0.90      0.90      1028
           8       0.87      0.84      0.85       974
           9       0.90      0.85      0.87      1009

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

